# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # TODO: increase to speed up training time?
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": 1, # TODO: weight higher!
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [5]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-23_08-20-39
['pgrep', 'tensorboard'] yielded -> b''


In [6]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-23_08-20__root.log
Starting train_vaegan main
Numpy random seed: 995370888
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-23_08-20-39/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 55.50
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 100)
encoder/dense_2/BiasAdd:0: (?, 100)
epsilon: (?, 100)
z: (?, 100)
input_z: (?, 100)
decoder/dec_dense1/BiasAdd:

<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam_1:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/moving_mean/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normaliz

<tf.Variable 'decoder/dec_conv4/kernel/Adam_1:0' shape=(4, 4, 4, 8, 16) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam:0' shape=(3, 3, 3, 1, 8) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam_1:0' shape=(3, 3, 3, 1, 8) dtype=fl

Memory Use (GB): 1.1470603942871094
Epoch: 0 / 201, Batch: 28 (0 / 3712), Elapsed time: 1.77 mins
Enc Loss = 745.15, KL Divergence = 117.14, Reconstruction Loss = 0.06, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.83 mins
Memory Use (GB): 1.2777137756347656
Epoch: 0 / 201, Batch: 29 (0 / 3840), Elapsed time: 1.83 mins
Enc Loss = 706.14, KL Divergence = 103.24, Reconstruction Loss = 0.06, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.90 mins
Memory Use (GB): 1.3127365112304688
Epoch: 0 / 201, Batch: 30 (0 / 3968), Elapsed time: 1.90 mins
Enc Loss = 679.17, KL Divergence = 101.77, Reconstruction Loss = 0.06, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 1.96 mins
Memory Use (GB): 1.2708168029785156
Epoch: 0 / 201, Batch: 31 (0 / 4096), Elapsed time: 1.96 mins
Enc Loss = 639.14, KL Divergence = 87.33, Reconstruction Loss = 0.06, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.0

...

Memory Use (GB): 1.4521102905273438
Epoch: 180 / 201, Batch: 36 (4736 / 5504), Elapsed time: 496.68 mins
Enc Loss = 152.19, KL Divergence = 47.53, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 496.74 mins
Memory Use (GB): 1.3916091918945312
Epoch: 180 / 201, Batch: 37 (4864 / 5504), Elapsed time: 496.74 mins
Enc Loss = 153.95, KL Divergence = 46.64, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 496.80 mins
Memory Use (GB): 1.40283203125
Epoch: 180 / 201, Batch: 38 (4992 / 5504), Elapsed time: 496.80 mins
Enc Loss = 149.89, KL Divergence = 46.81, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 496.87 mins
Memory Use (GB): 1.3899002075195312
Epoch: 180 / 201, Batch: 39 (5120 / 5504), Elapsed time: 496.87 mins
Enc Loss = 152.42, KL Divergence = 48.80, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = 

Memory Use (GB): 1.3704681396484375
Epoch: 181 / 201, Batch: 24 (3200 / 5504), Elapsed time: 498.63 mins
Enc Loss = 154.79, KL Divergence = 50.90, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 498.70 mins
Memory Use (GB): 1.3907318115234375
Epoch: 181 / 201, Batch: 25 (3328 / 5504), Elapsed time: 498.70 mins
Enc Loss = 150.28, KL Divergence = 48.62, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 498.76 mins
Memory Use (GB): 1.4390029907226562
Epoch: 181 / 201, Batch: 26 (3456 / 5504), Elapsed time: 498.76 mins
Enc Loss = 151.64, KL Divergence = 45.47, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 498.82 mins
Memory Use (GB): 1.4778861999511719
Epoch: 181 / 201, Batch: 27 (3584 / 5504), Elapsed time: 498.82 mins
Enc Loss = 158.59, KL Divergence = 48.59, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Lo

Memory Use (GB): 1.3924827575683594
Epoch: 182 / 201, Batch: 12 (1664 / 5504), Elapsed time: 500.59 mins
Enc Loss = 166.40, KL Divergence = 54.74, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 500.65 mins
Memory Use (GB): 1.452972412109375
Epoch: 182 / 201, Batch: 13 (1792 / 5504), Elapsed time: 500.65 mins
Enc Loss = 153.43, KL Divergence = 49.32, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 500.71 mins
Memory Use (GB): 1.4643783569335938
Epoch: 182 / 201, Batch: 14 (1920 / 5504), Elapsed time: 500.71 mins
Enc Loss = 148.11, KL Divergence = 46.41, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 500.78 mins
Memory Use (GB): 1.2794723510742188
Epoch: 182 / 201, Batch: 15 (2048 / 5504), Elapsed time: 500.78 mins
Enc Loss = 158.16, KL Divergence = 48.00, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Los

Epoch: 183, Elapsed Time: 502.54


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

